In [0]:
#@title Setup
!apt-get install xmlstarlet
!pip install dask
!pip install pandas
!pip install beautifulsoup4
!rm -rf out feed*

In [0]:
#@title Get links
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict
import dask
import dask.dataframe as dd
from dask.dataframe.utils import make_meta
from tqdm import tqdm
import os
import xml.etree.cElementTree as et
from pprint import pprint


STARTPAGE = 1 #@param {type:"number"}
ENDPAGE = 71 #@param {type:"number"}


WORKERS = 64 #@param {type:"number"}
dask.config.set(work_stealing=False)
dask.config.set(num_workers=WORKERS)




def getmcuredirect(url):
  try:
    if 'mcupaste.com' in url:
      r = requests.get(url)
      soup = BeautifulSoup(r.text, 'html5lib')
      return soup.select_one('#raw_content').get_text()
    else:
      return s
  except:
    return url 


def getlinks(row):
  try:
    soup = BeautifulSoup(row['content'], 'html5lib')
    links = soup.findAll('a')

    urls=[]
    for l in links:
      url = l['href']
      if not '//mcumovieshome.com' in url:
        urls.append(getmcuredirect(url))

    row['links'] = ' '.join(urls)
  except:
    pass
  pbar.update(1)
  return row


dfcols=['title','link','content']
df_xml = pd.DataFrame(columns=dfcols)

for i in range(STARTPAGE, ENDPAGE+1):
  r = requests.get("https://mcumovieshome.com/feed/?paged=%d"%(i))
  root = et.fromstring(r.content)
  rows = root.findall('.//item')
  xml_data = [[row.find('title').text, row.find('link').text, row.find('{http://purl.org/rss/1.0/modules/content/}encoded').text] 
            for row in rows]
  df_xml = df_xml.append(pd.DataFrame(xml_data, columns=dfcols))

df_xml['links']=''
pbar = tqdm(total=len(df_xml), ncols=80)

dd_xml = dd.from_pandas(df_xml, npartitions=WORKERS)
df_xml = dd_xml.apply(getlinks, axis=1, meta=make_meta(df_xml)).compute()
df_xml = df_xml.drop(columns='content')
  
df_xml.to_csv('mcumovies.csv', encoding='UTF-8', index=False)
print('\n\n\nCreate file mcumovies.csv\n\n(On the right, click on "Files")')